In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    database_name = '%s_upcache' % game
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{database_name}', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1648080000
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_growth_okr_dau_sql(game,start_time,end_time,frame_platform=None):
    frame_platform_sql = ''
    if frame_platform:
        frame_platform_sql = "AND acq.frame_platform = '%s'" % frame_platform
    return """
    SELECT master.day AS `day`,
    (SELECT ROUND(SUM(dau)/3)
    FROM %s_upcache.%s_sessions_daily_summary AS acq
    WHERE acq.day >= master.day - 3*86400
    AND acq.day < master.day
    %s
    AND acq.country_tier IN ('1','2')) AS `dau`
    FROM skynet.bh_daily_summary master
    WHERE master.day >= %d - 90*86400
    AND master.day < %d
    GROUP BY master.day;
    """ % (game, game, frame_platform_sql,start_time,end_time)

def get_retention_okr_sql(game,start_time,end_time,time_now):
    return """SELECT  1337274000 + 14*86400*(1+FLOOR((account_creation_time + 8*86400 - 1337274000)/(14*86400))) AS pvp_week_end,
    IF(country_tier IN ('1','2'), 'T12', 'T34') AS tier,
    IF(IFNULL(acquisition_campaign,'MISSING') LIKE '%s' OR acquisition_campaign LIKE '%s', 'Paid', 'Free') AS acquisition_type,
    COUNT(1) as N,
    ROUND(SUM(IF(toc_level >= 2,1,0)) / SUM(1), 2) AS `TOC L2`,
    ROUND(SUM(IF(`returned_24-48h`,1,0)) / SUM(1), 2) AS `1-day Return`,
    ROUND(SUM(IF(`returned_48-72h`,1,0)) / SUM(1), 2) AS `2-day Return`,
    ROUND(SUM(IF(`returned_72-96h`,1,0)) / SUM(1), 2) AS `3-day Return`,
    ROUND(SUM(IF(`returned_120-144h`,1,0)) / SUM(1), 2) AS `5-day Return`,
    ROUND(SUM(IF(`returned_168-192h`,1,0)) / SUM(1), 2) AS `7-day Return`
    FROM `%s_upcache`
    WHERE account_creation_time + 8*86400 >= %d - 14*86400
    AND account_creation_time + 8*86400 < LEAST(%d, %d)
    GROUP BY `pvp_week_end`, tier, acquisition_type
    HAVING pvp_week_end < %d;""" % ('%%_SRD','%%_GG',game,start_time,time_now,end_time,time_now)

def get_game_content_okr_damage_sql(game, start_time, end_time):
    return """SELECT brr.day AS `day`,
                 1337274000 + 7*86400*(1+FLOOR((brr.day - 1337274000)/(7*86400)))  AS pvp_week_end,
                 IF(MOD(FLOOR((brr.day - 1337274000)/(7*86400)),2)=1,'ONP','Immortal')  AS pvp_week_type,
                 -SUM(brr.damage_iron_water_amount) AS `current_qtr_dmg`
                 FROM `%s_battles_risk_reward_daily_summary` AS brr
                 WHERE brr.day >= %d + 1*86400
                 AND brr.day < %d
                 AND brr.townhall_level >= 5
                 GROUP BY `day`;""" % (game, start_time, end_time)

def get_game_content_okr_dmg_dau_sql(game, start_time, end_time):
    return """SELECT ses.day AS `day`, SUM(ses.dau) AS `dau`
                     FROM %s_sessions_daily_summary AS ses
                     WHERE (ses.townhall_level >= 5 AND ses.day >= %d + 1*86400 AND ses.day < %d)
                     GROUP BY `day`;""" % (game, start_time, end_time)

def get_game_content_okr_durable_spend_sql(game, start_time, end_time):
    return """SELECT cur.day+1*86400 AS day,
              SUM(cur.total_price) AS `current_qtr_gamebucks`,
              (SELECT SUM(prev.total_price)
              FROM `tr_store_daily_summary` AS prev
              WHERE prev.day = cur.day - 90*86400
              AND prev.currency = 'gamebucks'
              AND prev.townhall_level >= 5
              AND prev.country_tier IN ('1','2')
              AND prev.category in ('research','resource_boost','crafting','building_upgrade','enhancement')) as previous_qtr_gamebucks
              FROM `tr_store_daily_summary` AS cur
              WHERE cur.day >= %d
              AND cur.day < %d+86400
              AND cur.currency = 'gamebucks'
              AND cur.townhall_level >= 5
              AND cur.country_tier IN ('1','2')
              AND cur.category in ('research','resource_boost','crafting','building_upgrade','enhancement')
              GROUP BY day
              ORDER BY day ASC LIMIT 1000;""" % (start_time, end_time)

def get_game_content_okr_spend_supply_sql(game, start_time, end_time):
    return """SELECT cur.time+1*86400 AS `day`,
              SUM(cur.total_amount) AS current_qtr_supply,
              (SELECT SUM(prev.total_amount)
              FROM `tr_active_player_resource_levels` AS prev
              WHERE (prev.time + 90*86400) BETWEEN cur.time - 43200 AND cur.time + 43200
              AND prev.townhall_level >= 5
              AND prev.country_tier IN ('1','2')
              AND prev.resource = 'gamebucks') as previous_qtr_supply
              FROM `tr_active_player_resource_levels` AS cur
              WHERE cur.time >= %d
              AND cur.time < %d + 86400
              AND cur.townhall_level >= 5
              AND cur.country_tier IN ('1','2')
              AND cur.resource = 'gamebucks'
              GROUP BY `day`
              ORDER BY `day` ASC LIMIT 1000;
              """ % (start_time, end_time)

def get_maintenance_okr_ccl_dau_sql(game, start_time, end_time, level):
    return """SELECT main.day AS `day`,
              (SELECT SUM(prev.dau)
              FROM %s_sessions_daily_summary prev
              WHERE prev.day = main.day - 90*86400
              AND prev.townhall_level >= %d) AS `prevqtr`,
              SUM(main.dau) AS `thisqtr`
              FROM %s_sessions_daily_summary main
              WHERE main.day >= %d AND main.day < %d
              AND main.townhall_level >= %d
              GROUP BY `day` ORDER BY `day` ASC;""" % (game, level, game, start_time, end_time, level)

def get_maintenance_okr_receipts_sql(game, start_time, end_time, level):
    return """SELECT cred.day + 1*86400 AS `day`,
              SUM(cred.usd_receipts_cents)/100 AS current_qtr_usd,
              (SELECT SUM(prev.usd_receipts_cents)/100
              FROM `%s_credits_daily_summary` AS prev
              WHERE prev.day = cred.day - 90*86400
              AND prev.townhall_level >= %d) as previous_qtr_usd
              FROM `tr_credits_daily_summary` AS cred
              WHERE cred.day >= %d
              AND cred.day < %d
              AND cred.townhall_level >= %d
              GROUP BY `day`
              ORDER BY `day` ASC LIMIT 1000;""" % (game, level, start_time, end_time, level)

def get_maintenance_okr_receipts_per_spend_sql(game, start_time, end_time, level):
    return """SELECT timeline.day AS `day`,
              (SELECT SUM(rolling.usd_receipts_cents)
              FROM `%s_credits_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 AND rolling.day <= timeline.day
              AND rolling.townhall_level >= %d) AS `dollars`,
              (SELECT SUM(rolling.total_price)
              FROM `%s_store_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 AND rolling.day <= timeline.day
              AND rolling.townhall_level >= %d
              AND rolling.currency = 'gamebucks') AS `gamebucks`,
              (SELECT SUM(rolling.usd_receipts_cents)
              FROM `%s_credits_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 - 90*86400 AND rolling.day <= timeline.day - 90*86400
              AND rolling.townhall_level >= %d) AS `dollarsprev`,
              (SELECT SUM(rolling.total_price)
              FROM `%s_store_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 - 90*86400 AND rolling.day <= timeline.day - 90*86400
              AND rolling.townhall_level >= %d
              AND rolling.currency = 'gamebucks') AS `gamebucksprevqtr`
              FROM (SELECT sess.day AS day
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline LIMIT 999999;""" % (game, level, game, level, game, level, game, level, game, start_time, end_time)

def get_maintenance_okr_gold_stock_avg_sql(game, start_time, end_time, level):
    return """SELECT 86400*ROUND(time/86400) AS `day`,
              IF(SUM(num_players)>0,SUM(total_amount)/SUM(num_players),NULL) AS `average`
              FROM %s_active_player_resource_levels
              WHERE resource = 'gamebucks'
              AND time >= %d AND time < %d
              AND country_tier IN ('1' , '2')
              AND townhall_level >= %d
              GROUP BY `Day` ORDER BY `Day` ASC LIMIT 1000;""" % (game, start_time, end_time, level)

def get_maintenance_okr_toc_promotion_L2_sql(game, start_time, end_time, frame_mod):
    return """SELECT timeline.day AS `day`,
              SUM(IF(th2.user_id IS NOT NULL,1,0)) / SUM(1) AS `promotion`
              FROM (SELECT sess.day AS DAY
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d-90*86400
              AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline,
              (SELECT user_id, account_creation_time
              FROM `%s_upcache` AS up_raw
              WHERE up_raw.account_creation_time + 7*86400 >= %d - 90*86400
              AND up_raw.account_creation_time + 1*86400 < %d
              AND frame_platform %s) AS temp_up
              LEFT JOIN tr_townhall_at_time th2 ON (th2.user_id = temp_up.user_id
                                                    AND th2.townhall_level = 2 -- threshold TOC level
                                                    AND th2.time >= temp_up.account_creation_time
                                                    AND th2.time < temp_up.account_creation_time+1*86400)
              WHERE temp_up.account_creation_time >= timeline.day - 7*86400
              AND temp_up.account_creation_time < timeline.day - 1*86400
              GROUP BY timeline.day
              ORDER BY timeline.day ASC;""" % (game, start_time, end_time, game, start_time, end_time, frame_mod)

def get_maintenance_okr_toc_promotion_L3_sql(game, start_time, end_time, frame_mod):
    return """SELECT timeline.day AS `day`,
              (SELECT SUM(IF(EXISTS(SELECT 1
                                    FROM %s_townhall_at_time th2
                                    WHERE th2.user_id = th.user_id
                                    AND th2.townhall_level = 3
                                    AND th2.time <= th.time+10*86400),1,0)) / COUNT(1)
              FROM %s_townhall_at_time th INNER JOIN %s_upcache u
              WHERE th.townhall_level = 2
              AND th.time >= timeline.day - 20*86400
              AND th.time < timeline.day - 10*86400
              AND u.user_id = th.user_id
              AND u.frame_platform %s) AS `promotion`
              FROM (SELECT sess.day AS day
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d - 90*86400
              AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline;""" % (game, game, game, frame_mod, game, start_time, end_time)

In [2]:
# prepare data for all OKRs

# universal setup for OKR data
time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

In [3]:
# prepare growth OKR data
game_frames = {'tr':['fb','bh','k2'],'fs':['bh']}
growth_okr_dau = {}
engine = get_engine('tr')
for game in ['tr','dv','mf2','bfm','mf','fs']:
    if game in game_frames:
        for frame in game_frames[game]:
            dau_sql = get_growth_okr_dau_sql(game,start_time,end_time,frame)
            growth_okr_dau_data = pd.read_sql(dau_sql, engine)
            if 'day' not in growth_okr_dau:
                growth_okr_dau['day'] = []
                for day in growth_okr_dau_data['day']:
                    growth_okr_dau['day'].append(day)
            key = game.upper() + ' (' +  frame.upper() + ')'
            growth_okr_dau[key] = []
            for day in growth_okr_dau_data['dau']:
                growth_okr_dau[key].append(day)
            if 'total' not in growth_okr_dau:
                growth_okr_dau['total'] = []
                for day in growth_okr_dau_data['dau']:
                    growth_okr_dau['total'].append(day)
            else:
                for i, day in enumerate(growth_okr_dau_data['dau']):
                    growth_okr_dau['total'][i] += day
    else:
        dau_sql = get_growth_okr_dau_sql(game,start_time,end_time)
        growth_okr_dau_data = pd.read_sql(dau_sql, engine)
        if 'day' not in growth_okr_dau:
            dau['day'] = []
            for day in growth_okr_dau_data['day']:
                dau['day'].append(day)
        key = game.upper()
        growth_okr_dau[key] = []
        for day in growth_okr_dau_data['dau']:
            growth_okr_dau[key].append(day)
        if 'total' not in growth_okr_dau:
            dau['total'] = []
            for day in growth_okr_dau_data['dau']:
                growth_okr_dau['total'].append(day)
        else:
            for i, day in enumerate(growth_okr_dau_data['dau']):
                growth_okr_dau['total'][i] += day

In [4]:
# prepare retention OKR data
retention_okr_data = {'tr':{},'dv':{}}
retention_okr_weeks = []
for game in ('tr','dv'):
    engine = get_engine(game)
    retention_okr_sql = get_retention_okr_sql(game,start_time,end_time,time_now)
    with engine.connect() as con:
        rs = con.execute(retention_okr_sql)
        for row in rs:
            this_data = {}
            this_data['week'] = int(row[0])
            if int(row[0]) not in retention_okr_weeks:
                retention_okr_weeks.append(int(row[0]))
            this_data['tier'] = row[1]
            this_data['type'] = row[2]
            this_data['n'] = str(row[3])
            this_data['tocL2'] = '{0:.0%}'.format(row[4])
            this_data['1D'] = '{0:.0%}'.format(row[5])
            this_data['2D'] = '{0:.0%}'.format(row[6])
            this_data['3D'] = '{0:.0%}'.format(row[7])
            this_data['5D'] = '{0:.0%}'.format(row[8])
            this_data['7D'] = '{0:.0%}'.format(row[9])
            data_index = str(int(row[0])) + row[1]
            retention_okr_data[game][data_index] = this_data

In [5]:
# prepare game content OKR data
engine = get_engine('tr')
game_content_okr_dmg = {'day':[],'pvp_week_end':[], 'pvp_week_type':[],'current_qtr_dmg':[],'dau':[]}
game_content_okr_dmg_sql = get_game_content_okr_damage_sql('tr',start_time,end_time)
game_content_okr_dmg_data = pd.read_sql(game_content_okr_dmg_sql, engine)
for day in game_content_okr_dmg_data['day']:
    game_content_okr_dmg['day'].append(day)
for day in game_content_okr_dmg_data['pvp_week_end']:
    game_content_okr_dmg['pvp_week_end'].append(day)
for day in game_content_okr_dmg_data['pvp_week_type']:
    game_content_okr_dmg['pvp_week_type'].append(day)
for day in game_content_okr_dmg_data['current_qtr_dmg']:
    game_content_okr_dmg['current_qtr_dmg'].append(day)
game_content_okr_dmg_dau_sql = get_game_content_okr_dmg_dau_sql('tr',start_time,end_time)
game_content_okr_dmg_dau_data = pd.read_sql(game_content_okr_dmg_dau_sql, engine)
for day in game_content_okr_dmg_dau_data['dau']:
    game_content_okr_dmg['dau'].append(day)
    
game_content_okr_durable_spend_sql = get_game_content_okr_durable_spend_sql('tr',start_time,end_time)
game_content_okr_durable_supply_sql = get_game_content_okr_spend_supply_sql('tr',start_time,end_time)
game_content_okr_durable_spend_data = pd.read_sql(game_content_okr_durable_spend_sql, engine)
game_content_okr_durable_supply_data = pd.read_sql(game_content_okr_durable_supply_sql, engine)
game_content_okr_last_q_total_spend_sql = get_game_content_okr_durable_spend_sql('tr',start_time - (86400 * 7 * 14), end_time - (86400 * 7 * 14))
game_content_okr_last_q_total_supply_sql = get_game_content_okr_spend_supply_sql('tr',start_time - (86400 * 7 * 14), end_time - (86400 * 7 * 14))
game_content_okr_last_q_spend_data = pd.read_sql(game_content_okr_last_q_total_spend_sql, engine)
game_content_okr_last_q_supply_data = pd.read_sql(game_content_okr_last_q_total_supply_sql, engine)

game_content_okr_durable_spend = {'day':[],'cur_spend':[], 'prev_spend':[],'cur_supply':[],'prev_supply':[]}
for day in game_content_okr_durable_spend_data['day']:
    game_content_okr_durable_spend['day'].append(day)
for day in game_content_okr_durable_spend_data['current_qtr_gamebucks']:
    game_content_okr_durable_spend['cur_spend'].append(day)
for day in game_content_okr_durable_spend_data['previous_qtr_gamebucks']:
    game_content_okr_durable_spend['prev_spend'].append(day)
for day in game_content_okr_durable_supply_data['current_qtr_supply']:
    game_content_okr_durable_spend['cur_supply'].append(day)
for day in game_content_okr_durable_supply_data['previous_qtr_supply']:
    game_content_okr_durable_spend['prev_supply'].append(day)
game_content_okr_last_q_spend = {'day':[],'spend':[], 'supply':[]}
for day in game_content_okr_last_q_spend_data['day']:
    game_content_okr_last_q_spend['day'].append(day)
for day in game_content_okr_last_q_spend_data['current_qtr_gamebucks']:
    game_content_okr_last_q_spend['spend'].append(day)
for day in game_content_okr_last_q_supply_data['current_qtr_supply']:
    game_content_okr_last_q_spend['supply'].append(day)

In [6]:
# prepare maintenance OKR data

engine = get_engine('tr')
maintenance_okr_ccl_dau_sql = get_maintenance_okr_ccl_dau_sql('tr',start_time,end_time,5)
maintenance_okr_ccl_dau_data = pd.read_sql(maintenance_okr_ccl_dau_sql, engine)
maintenance_okr_receipts_sql = get_maintenance_okr_receipts_sql('tr',start_time,end_time,5)
maintenance_okr_receipts_data  = pd.read_sql(maintenance_okr_receipts_sql, engine)
maintenance_okr_receipts_per_spend_sql = get_maintenance_okr_receipts_per_spend_sql('tr',start_time,end_time,5)
maintenance_okr_receipts_per_spend_data  = pd.read_sql(maintenance_okr_receipts_per_spend_sql, engine)
maintenance_okr_gold_stock_avg_sql = get_maintenance_okr_gold_stock_avg_sql('tr', start_time, end_time, 5)
maintenance_okr_gold_stock_avg_data  = pd.read_sql(maintenance_okr_gold_stock_avg_sql, engine)
maintenance_okr_toc_promotion_L2_sql = get_maintenance_okr_toc_promotion_L2_sql('tr', start_time, end_time, '!= "bh"')
maintenance_okr_toc_promotion_L2_data = pd.read_sql(maintenance_okr_toc_promotion_L2_sql, engine)
maintenance_okr_toc_promotion_L2_electron_sql = get_maintenance_okr_toc_promotion_L2_sql('tr', start_time, end_time, '= "bh"')
maintenance_okr_toc_promotion_L2_electron_data = pd.read_sql(maintenance_okr_toc_promotion_L2_electron_sql, engine)
maintenance_okr_toc_promotion_L3_sql = get_maintenance_okr_toc_promotion_L3_sql('tr', start_time, end_time, '!= "bh"')
maintenance_okr_toc_promotion_L3_data = pd.read_sql(maintenance_okr_toc_promotion_L3_sql, engine)
maintenance_okr_toc_promotion_L3_electron_sql = get_maintenance_okr_toc_promotion_L3_sql('tr', start_time, end_time, '= "bh"')
maintenance_okr_toc_promotion_L3_electron_data = pd.read_sql(maintenance_okr_toc_promotion_L3_electron_sql, engine)

In [9]:

# output all OKRs
print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

# Growth OKR display
print('All Game Tier 1/2 DAU, average of trailing 3 days')
growth_okr_boundaries = get_quarter_time_boundaries(time_now)
growth_okr_boundaries['start'] = growth_okr_boundaries['start'] + 86400*7
for i, day in enumerate(growth_okr_dau['day']):
    if day < growth_okr_boundaries['start'] or day > growth_okr_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        print(datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d') + ': ' + str(int(growth_okr_dau['total'][i])))
print('Change since last week (if any)')
game_frames = {'tr':['fb','bh','k2'],'fs':['bh']}
for i, day in enumerate(growth_okr_dau['day']):
    if day < growth_okr_boundaries['start'] or day > growth_okr_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d') + ': '
        total_changes = 0
        for game in ['tr','dv','mf2','bfm','mf','fs']:
            if game in game_frames:
                for frame in game_frames[game]:
                    key = game.upper() + ' (' +  frame.upper() + ')'
                    this_week = int(growth_okr_dau[key][i])
                    last_week = int(growth_okr_dau[key][i-1])
                    change = this_week - last_week
                    if change != 0:
                        if change > 0:
                            output_line += ' %s: +%d,' % (key.replace('K2','KG'), change)
                        else:
                            output_line += ' %s: %d,' % (key.replace('K2','KG'), change)
                        total_changes += 1
            else:
                key = game.upper()
                this_week = growth_okr_dau[key][i]
                last_week = growth_okr_dau[key][i-1]
                change = this_week - last_week
                if change != 0:
                    if change > 0:
                        output_line += ' %s: +%d,' % (key.replace('K2','KG'), change)
                    else:
                        output_line += ' %s: %d,' % (key.replace('K2','KG'), change)
                    total_changes += 1
        if total_changes == 0:
            output_line += 'No Change'
        print(output_line)
print('')

# retention OKR display
retention_okr_weeks.sort()
for game in ('tr','dv'):
    print('%s Retention (for accounts 8+ days old) * only updated every 2 weeks' % game.upper())
    header = "{0:12} {1:5} {2:6} {3:6} {4:8} {5:8} {6:8} {7:8} {8:8} {9:8}".format('Week', 'Tier', 'Type', 'N', 'TOC L2', '1-day', '2-day','3-day','5-day','7-day')
    print(header)
    for week in retention_okr_weeks:
        for tier in ('T12','T34'):
            key = str(week) + tier
            if key not in retention_okr_data[game]: continue
            this_data = retention_okr_data[game][key]
            formatted_week = formatted_date = datetime.datetime.utcfromtimestamp(week).strftime('%Y-%m-%d')
            this_row = "{0:12} {1:5} {2:6} {3:6} {4:8} {5:8} {6:8} {7:8} {8:8} {9:8}".format(formatted_week, this_data['tier'],this_data['type'],this_data['n'],this_data['tocL2'],
                                                                                           this_data['1D'],this_data['2D'],this_data['3D'],this_data['5D'],this_data['7D'])
            print(this_row)
    print('')
    
# game content OKR display

print('TR Cumulative Damage Suffered Per DAU (TOC L5+, x10000)')
boundaries = get_quarter_time_boundaries(time.time())
boundaries['start'] = boundaries['start'] + 86400*7
game_content_okr_total_dmg = {}
game_content_okr_weeks = []
for i, day in enumerate(game_content_okr_dmg['day']):
    week_int = int(game_content_okr_dmg['pvp_week_end'][i])
    if week_int not in game_content_okr_total_dmg:
        game_content_okr_total_dmg[week_int] = {'pvp_week_type':game_content_okr_dmg['pvp_week_type'][i], 'total':0}
    game_content_okr_total_dmg[week_int]['total'] += 0.0001 * game_content_okr_dmg['current_qtr_dmg'][i] / game_content_okr_dmg['dau'][i]
    if week_int not in game_content_okr_weeks:
        game_content_okr_weeks.append(week_int)
game_content_okr_total_onp = 0
game_content_okr_total_immortal = 0
header = "{0:12} {1:8} {2:10} {3:6}".format('Week', 'Damage', 'Type', 'Cumulative Damage')
print(header)
for week in game_content_okr_weeks:
    game_content_okr_week_data = game_content_okr_total_dmg[week]
    week_type = game_content_okr_week_data['pvp_week_type']
    this_week_total = math.ceil(game_content_okr_week_data['total'])
    output_total = 0
    if week_type == 'ONP':
        game_content_okr_total_onp += this_week_total
        output_total = game_content_okr_total_onp
    elif week_type == 'Immortal':
        game_content_okr_total_immortal += this_week_total
        output_total = game_content_okr_total_immortal
    formatted_date = datetime.datetime.utcfromtimestamp(week).strftime('%Y-%m-%d')
    if week < boundaries['start'] or week > boundaries['end']: continue
    output = "{0:12} {1:8} {2:10} {3:6}".format(formatted_date, str(this_week_total), week_type, str(output_total))
    print(output)
print('')
print('')


game_content_okr_last_q_total = 0
for i, day in enumerate(game_content_okr_last_q_spend['day']):
    game_content_okr_last_q_total += game_content_okr_last_q_spend['spend'][i] / game_content_okr_last_q_spend['supply'][i]
game_content_okr_last_q_daily_goal = 1.1 * game_content_okr_last_q_total / 13 / 7
print('TR Cumulative Durable Gamebuck Spend per Supply (TOC L5+) Tier 1/2')
header = "{0:12} {1:8} {2:10} {3:6}".format('Week', 'Target', 'Spend', 'Last Quarter')
print(header)
spend = 0
target = 0
prev_q_spend = 0
for i, day in enumerate(game_content_okr_durable_spend['day']):
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    if i < len(game_content_okr_durable_spend['cur_spend']) and i < len(game_content_okr_durable_spend['cur_supply']):
        spend += game_content_okr_durable_spend['cur_spend'][i] / game_content_okr_durable_spend['cur_supply'][i]
    if i < len(game_content_okr_durable_spend['prev_spend']) and i < len(game_content_okr_durable_spend['prev_supply']):
        prev_q_spend += game_content_okr_durable_spend['prev_spend'][i] / game_content_okr_durable_spend['prev_supply'][i]
    target += game_content_okr_last_q_daily_goal
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8} {2:10} {3:6}".format(output_date, str(round(target, 2)), str(round(spend, 2)), str(round(prev_q_spend, 2)))
        print(output_line)
        
print('')
print('')
# maintenance OKR display
boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries['start'] = diplay_boundaries['start'] + 86400*7
print('TR CCL5+ DAU vs. previous quarter end')
header = "{0:12} {1:8}".format('Week', 'DAU')
print(header)
for i, day in enumerate(maintenance_okr_ccl_dau_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    maintenance_okr_ccl_dau = maintenance_okr_ccl_dau_data['thisqtr'][i]
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(int(maintenance_okr_ccl_dau)))
        print(output_line)
print('')
print('TR Monetization: Cumulative Sum of Receipts per CCL5+ DAU')
header = "{0:12} {1:8}".format('Week', 'Receipts/DAU')
print(header)
maintenance_okr_ccl_receipts = 0.0
for i, day in enumerate(maintenance_okr_receipts_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    maintenance_okr_ccl_dau = maintenance_okr_ccl_dau_data['thisqtr'][i]
    maintenance_okr_ccl_receipts += maintenance_okr_receipts_data['current_qtr_usd'][i] / maintenance_okr_ccl_dau
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(round(maintenance_okr_ccl_receipts, 2)))
        print(output_line)
print('')
print('TR USD Receipts per gamebuck')
header = "{0:12} {1:8}".format('Week', 'Receipts/Gamebuck')
print(header)
for i, day in enumerate(maintenance_okr_receipts_per_spend_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_receipts = maintenance_okr_receipts_per_spend_data['dollars'][i] / maintenance_okr_receipts_per_spend_data['gamebucks'][i]
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(round(output_receipts, 2)))
        print(output_line)
print('')
print('TR Gold Stock Average CCL5+ Tier 1/2 vs. previous quarter')
header = "{0:12} {1:8}".format('Week', 'Average Gold Stock')
print(header)
for i, day in enumerate(maintenance_okr_receipts_per_spend_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(int(maintenance_okr_gold_stock_avg_data['average'][i])))
        print(output_line)

print('')
print('TR TOC L2 promotion within 1d (non-Electron)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L2_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L2_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

print('')
print('TR TOC L2->L3 promotion within 10d (non-Electron)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L3_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L3_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

print('')
print('TR TOC L2 promotion within 1d (Electron only)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L2_electron_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L2_electron_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

print('')
print('TR TOC L2->L3 promotion within 10d (Electron)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L3_electron_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L3_electron_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

Dashboard updated Sat, 22 Jan 2022 at 17:41:32 UTC
All Game Tier 1/2 DAU, average of trailing 3 days
2021-12-30: 2735
2022-01-06: 2778
2022-01-13: 2776
2022-01-20: 2779
Change since last week (if any)
2021-12-30:  TR (FB): +12, TR (BH): -1, TR (KG): +2, MF2: +2, BFM: +3, MF: +6, FS (BH): +4,
2022-01-06:  TR (FB): -6, TR (BH): -1, DV: +3, MF2: +11, BFM: +1, MF: +3, FS (BH): +1,
2022-01-13:  TR (FB): +8, DV: +4, MF2: -1, BFM: -2, MF: +5, FS (BH): +2,
2022-01-20:  TR (FB): +1, TR (KG): +1, DV: -1, MF2: +1, BFM: +1, MF: -3, FS (BH): +3,

TR Retention (for accounts 8+ days old) * only updated every 2 weeks
Week         Tier  Type   N      TOC L2   1-day    2-day    3-day    5-day    7-day   
2021-12-16   T12   Free   11     27%      18%      9%       9%       9%       0%      
2021-12-16   T34   Free   34     59%      18%      21%      18%      18%      18%     
2021-12-30   T12   Free   33     45%      12%      9%       15%      12%      12%     
2021-12-30   T34   Free   61     69%      2